![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### ODPi Egeria Hands-On Lab
# Welcome to the Understanding Methods for Restricting Asset Access Lab

## Introduction

ODPi Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogues and platforms together so they can share information about data and technology (called metadata).

In this hands-on lab you will get a chance to explore the importance of applying data redaction where Metadata assets have restricted access using Palisade with Egeria

There are three methods we will go through to view the effects on data access, these are using:


- Simple Access Control : Access Control List (ACL)
- Policy Based Access : Glossary Based Policy Control(GBPC)
- Context Sensitive Access : Palisade (Scalable Data Access Policy Management and Enforcement : https://github.com/gchq/Palisade)

The three methods highlight the different ways data access and redaction can be performed.  With all three methods the user Callie is accessing the same assets, with each method above the access control becomes increasingly more fine grained:

- **ACL**, users to have access to a file or not and redaction is not possible, further more if the file is coppied and the copy does not have appropriate file access controls applied to it then the data is not protected.  With the scenario Callie would be able to see all data.

With the next two options it should be noted that the data if in a data lake and access is virtulised you can apply an enforcement point between the user and the data.  This allow the organisation to use policy based access control which are the two options below. 

- **GBPC**, Here Glossary based policies are utilised to control access as used in Apache Ranger.  This enables access security to be applied to file with redaction of certain elements this is profiled against the individual users identity, so Callie may be able to see more/less that her co-workers, however, two data sets are required so different redactions can be applied.  
- **Contect Sensitive Access**, enables Callie access to dynamically change based on the context of her work. Meaning there is NO duplication of data as the access is modified when she changes between project and data requests.  To perform this with Egeria we have configure this demonstration to use Palisade.


## The scenario

Callie Quartile is a data scientist at Coco Pharmaceuticals. She is responsible for analysing data for HR and the Clinical Trials team. 

Callie has been asked to provide analytics for two different projects:

- a staff salary review that identifies any pay biases
- identify staff elligble for a 5 year anniversay health screening project

The data that Callie will access contains sensitive and personal data which she is not authorised to view, such as Salary, Data of Birth, Employee Name and so on.  In this notebook you will learn how to redact data elements, so Callie can only view the data that is essential to each project in a way which does not violate her level of data access.

There are many situations where data needs to be redacted for different members of staff, based on their role, access, security clearance and so on.  In the first report it is inappropriate for Callie to view her colleagues salary details along with their names, employee number or other items which identify them.  If certain details are redacted then Callie may see the salaries with no knowledge of who the data pertains to.  In the analytics she provides for the Marketing team Callie will see a different set of data which is redacted based on a different set of rules based on context.


![Callie Quartile](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/callie-quartile.png)

Callie's userId is `calliequartile`.

In [4]:
calliesUserId = "calliequartile"

The two projects (HR and Marketing) used overlapping data.  Which dependant on the analytics and research that Callie is performing require that different elements of data are redacted.

**HR Sallary Bias Review Project Data** 
The data access for this project requires consist of employee records each with a number of fields. Certain fields contain PII information and require special privaliges to be accessed.  To perform the Analytics and research, Callie needs to access PII data such as Salary, Gender, Nationality, Work Location and so on. If the fields that identify the individual are redacted then Callie can perform the Analytics and Research.

**HR Health Screening Project Data**
In this instance Callie is still restricted from seeing elements of her collegues PII data, however, these are different items from the previous analysis she performed.  In this instace she will have access to employee, name, id, age, length of service and address; Access to financial infoamtion in this instance is restricted.

# Setting up

Coco Pharmaceuticals make widespread use of ODPi Egeria for tracking and managing their data and related assets.
Figure 1 below shows the metadata servers and the platforms that are hosting them.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

In [3]:
import os

corePlatformURL     = os.environ.get('corePlatformURL','http://localhost:8080') 
dataLakePlatformURL = os.environ.get('dataLakePlatformURL','http://localhost:8081') 
devPlatformURL      = os.environ.get('devPlatformURL','http://localhost:8082')

Callie is using the research team's metadata server called `cocoMDS3`. This server is hosted on the Core OMAG Server Platform.

In [ ]:
server            = "cocoMDS3"
serverPlatformURL = corePlatformURL

The following request checks that this server is running.

In [ ]:
import requests
import pprint
import json

adminUserId = "garygeeke"

isServerActiveURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + server + "/status"

print (" ")
print ("GET " + isServerActiveURL)
print (" ")

response = requests.get(isServerActiveURL)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

serverStatus = response.json().get('active')
if serverStatus == True:
    print("Server " + server + " is active - ready to begin")
else:
    print("Server " + server + " is down - start it before proceeding")


----
The next set of code sets up the asset - it is subject to change.

In [ ]:
assetOwnerURL = serverPlatformURL + '/servers/' + server + '/open-metadata/access-services/asset-owner/users/' + calliesUserId 
createAssetURL = assetOwnerURL + '/assets/csv-files'
print (createAssetURL)

jsonHeader = {'content-type':'application/json'}
body = {
	"class" : "NewFileAssetRequestBody",
	"displayName" : "Drop Foot Clinical Trial Patients",
	"description" : "List of patients registered for the drop foot clinical trial.",
	"fullPath" : "file://secured/research/clinical-trials/drop-foot/Patients.csv"
}

response=requests.post(createAssetURL, json=body, headers=jsonHeader)

response.json()

In [ ]:
getAssetsURL = serverAssetOwnerURL + '/assets/by-name?startFrom=0&pageSize=50'
searchString="*Patient*"

print (" ")
print ("GET " + getAssetsURL)
print ("{ " + searchString + " }")
print (" ")

response=requests.post(getAssetsURL, data=searchString)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

if response.json().get('assets'):
    if len(response.json().get('assets')) == 1:
        print ("1 asset found")
    else:
        print (str(len(response.json().get('assets'))) + " assets found")
else:
    print ("No assets found")